In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd '/content/gdrive/MyDrive/IOT Intrusion Detection'

/content/gdrive/MyDrive/IOT Intrusion Detection


**Import The Libraries**

In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
import tensorflow as tf

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.layers import LSTM, GRU, SimpleRNN

In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 10.4 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce

**Import The Dataset**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/IOT Intrusion Detection/IoT Network Intrusion Dataset.csv')

Analyze the dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 86 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625783 non-null  object 
 1   Src_IP             625783 non-null  object 
 2   Src_Port           625783 non-null  int64  
 3   Dst_IP             625783 non-null  object 
 4   Dst_Port           625783 non-null  int64  
 5   Protocol           625783 non-null  int64  
 6   Timestamp          625783 non-null  object 
 7   Flow_Duration      625783 non-null  int64  
 8   Tot_Fwd_Pkts       625783 non-null  int64  
 9   Tot_Bwd_Pkts       625783 non-null  int64  
 10  TotLen_Fwd_Pkts    625783 non-null  float64
 11  TotLen_Bwd_Pkts    625783 non-null  float64
 12  Fwd_Pkt_Len_Max    625783 non-null  float64
 13  Fwd_Pkt_Len_Min    625783 non-null  float64
 14  Fwd_Pkt_Len_Mean   625783 non-null  float64
 15  Fwd_Pkt_Len_Std    625783 non-null  float64
 16  Bw

In [ ]:
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:48 AM,141,0,3,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04/09/2019 03:58:17 AM,151,0,2,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/09/2019 01:41:18 AM,153,2,1,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg


**Drop the unnecessary columns**

In [ ]:
df = df.drop(['Flow_ID','Src_IP','Dst_IP', 'Timestamp'], axis = 1)

**drop the infinite values**

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(axis=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625415 non-null  int64  
 1   Dst_Port           625415 non-null  int64  
 2   Protocol           625415 non-null  int64  
 3   Flow_Duration      625415 non-null  int64  
 4   Tot_Fwd_Pkts       625415 non-null  int64  
 5   Tot_Bwd_Pkts       625415 non-null  int64  
 6   TotLen_Fwd_Pkts    625415 non-null  float64
 7   TotLen_Bwd_Pkts    625415 non-null  float64
 8   Fwd_Pkt_Len_Max    625415 non-null  float64
 9   Fwd_Pkt_Len_Min    625415 non-null  float64
 10  Fwd_Pkt_Len_Mean   625415 non-null  float64
 11  Fwd_Pkt_Len_Std    625415 non-null  float64
 12  Bwd_Pkt_Len_Max    625415 non-null  float64
 13  Bwd_Pkt_Len_Min    625415 non-null  float64
 14  Bwd_Pkt_Len_Mean   625415 non-null  float64
 15  Bwd_Pkt_Len_Std    625415 non-null  float64
 16  Fl

**Cehck the unique value of Label , Cat and Sub_cat Columns**

In [ ]:
df[['Label','Cat','Sub_Cat']].nunique()

Label      2
Cat        5
Sub_Cat    9
dtype: int64

**Drop the column those have only one unique value**

In [ ]:
redundant_feat = list(df.loc[:, df.nunique() == 1].columns)

In [ ]:
df = df.drop(redundant_feat, axis = 1)

**Check the dataset**

In [ ]:
df.shape

(625415, 72)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 72 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625415 non-null  int64  
 1   Dst_Port           625415 non-null  int64  
 2   Protocol           625415 non-null  int64  
 3   Flow_Duration      625415 non-null  int64  
 4   Tot_Fwd_Pkts       625415 non-null  int64  
 5   Tot_Bwd_Pkts       625415 non-null  int64  
 6   TotLen_Fwd_Pkts    625415 non-null  float64
 7   TotLen_Bwd_Pkts    625415 non-null  float64
 8   Fwd_Pkt_Len_Max    625415 non-null  float64
 9   Fwd_Pkt_Len_Min    625415 non-null  float64
 10  Fwd_Pkt_Len_Mean   625415 non-null  float64
 11  Fwd_Pkt_Len_Std    625415 non-null  float64
 12  Bwd_Pkt_Len_Max    625415 non-null  float64
 13  Bwd_Pkt_Len_Min    625415 non-null  float64
 14  Bwd_Pkt_Len_Mean   625415 non-null  float64
 15  Bwd_Pkt_Len_Std    625415 non-null  float64
 16  Fl

**Traffic Category will be our target variable**

In [ ]:
df['Cat'].value_counts()

Mirai                415309
Scan                  75265
DoS                   59391
Normal                40073
MITM ARP Spoofing     35377
Name: Cat, dtype: int64

In [ ]:
df['Label'].value_counts()

Anomaly    585342
Normal      40073
Name: Label, dtype: int64

In [ ]:
df['Sub_Cat'].value_counts()

Mirai-UDP Flooding       183189
Mirai-Hostbruteforceg    121178
DoS-Synflooding           59391
Mirai-HTTP Flooding       55818
Mirai-Ackflooding         55124
Scan Port OS              53073
Normal                    40073
MITM ARP Spoofing         35377
Scan Hostport             22192
Name: Sub_Cat, dtype: int64

These 5 classes we will be attempting to classify in our Classification Problem

**Apply Label Encoding on Categry column**

In [ ]:
labelencoder = LabelEncoder()
df['Sub_Cat']= labelencoder.fit_transform(df['Sub_Cat'])

**We have 69 independent features.**

**Category will be our target variable**

In [ ]:
X = df.iloc[:,0:69]
y = df['Sub_Cat']

In [ ]:
y.head()

0    2
1    0
2    8
3    4
4    4
Name: Sub_Cat, dtype: int64

**use to_categorical method to Converts a class vector (integers) to binary class mat**rix.

In [ ]:
y = tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)

In [ ]:
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
d1= pd.DataFrame(y)

In [ ]:
d1.head()

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


**Encoding Categorical Variables**

We have three Categorical Variables which are Src_Port, Dst_Port and Protocol
We need to encode this categorical data

We cannot do Label Encoding because categorical data Src_Port and Dst_Port are basically port number and thus they are categories only, there is no order between them

In [ ]:
X[['Src_Port','Dst_Port','Protocol']].nunique()

Src_Port    11205
Dst_Port     1034
Protocol        3
dtype: int64

As Categorical Data has too many categories we cannot do OneHotEncoding, as it will increase dimensionality of our feature data too much

In [ ]:
X['Src_Port'].unique()

array([10000,  2179, 52727, ..., 63997, 51806,  4656])

Thus we will be using Binary Encoding

In [ ]:
encoder = ce.BinaryEncoder(cols = ['Src_Port','Dst_Port','Protocol'])

In [ ]:
X = encoder.fit_transform(X)

In [ ]:
X.shape

(625415, 93)

In [ ]:
X.head()

,Src_Port_0,Src_Port_1,Src_Port_2,Src_Port_3,Src_Port_4,Src_Port_5,Src_Port_6,Src_Port_7,Src_Port_8,Src_Port_9,...,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,0,0,0,0,0,0,0,0,0,0,...,-1,1,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0
1,0,0,0,0,0,0,0,0,0,0,...,14600,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0
2,0,0,0,0,0,0,0,0,0,0,...,1869,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0
3,0,0,0,0,0,0,0,0,0,0,...,1869,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0
4,0,0,0,0,0,0,0,0,0,0,...,-1,2,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0


**Split the data into Training and Testing**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 100)

Check the shape 

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(500332, 93)
(500332, 9)
(125083, 93)
(125083, 9)


**Apply the Normalization on X data**

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train.shape

(500332, 93)

In [ ]:
X_test.shape

(125083, 93)

**Create ANN Model**

In [ ]:
# ANN MODEL
ann = Sequential()
ann.add(Dense(units=128, input_dim=93, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=256, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=512, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=256, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=128, activation='relu'))
ann.add(Dropout(0.5))
# ann.add(Dense(units=4096,activation="relu"))
# ann.add(Dense(units=4096,activation="relu"))
ann.add(Dense(units=y_train.shape[1], activation='softmax'))

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [ ]:
ann.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               12032     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 256)               33024     
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_20 (Dense)            (None, 512)               131584    
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 256)              

In [ ]:
# ann.fit(X_train, y_train)

15636/15636 [==============================] - 148s 9ms/step - loss: 0.1593 - accuracy: 0.9481


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("Resnet50.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early_stop = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
callbacks_list = [early_stop, checkpoint]

In [ ]:
history = ann.fit(X_train, y_train,verbose=2,epochs=30,batch_size=64,callbacks = callbacks_list)

Epoch 1/30

Epoch 1: accuracy improved from -inf to 0.76849, saving model to Resnet50.h5
7818/7818 - 25s - loss: 0.4811 - accuracy: 0.7685 - 25s/epoch - 3ms/step
Epoch 2/30

Epoch 2: accuracy improved from 0.76849 to 0.76886, saving model to Resnet50.h5
7818/7818 - 24s - loss: 0.4809 - accuracy: 0.7689 - 24s/epoch - 3ms/step
Epoch 3/30

Epoch 3: accuracy did not improve from 0.76886
7818/7818 - 23s - loss: 0.4898 - accuracy: 0.7685 - 23s/epoch - 3ms/step
Epoch 4/30

Epoch 4: accuracy did not improve from 0.76886
7818/7818 - 23s - loss: 0.4824 - accuracy: 0.7685 - 23s/epoch - 3ms/step
Epoch 5/30

Epoch 5: accuracy did not improve from 0.76886
7818/7818 - 23s - loss: 0.4824 - accuracy: 0.7683 - 23s/epoch - 3ms/step
Epoch 6/30

Epoch 6: accuracy improved from 0.76886 to 0.76896, saving model to Resnet50.h5
7818/7818 - 23s - loss: 0.4836 - accuracy: 0.7690 - 23s/epoch - 3ms/step
Epoch 7/30

Epoch 7: accuracy did not improve from 0.76896
7818/7818 - 23s - loss: 0.4835 - accuracy: 0.7683 - 2

In [ ]:
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,confusion_matrix)

In [ ]:
ann.evaluate(X_test, y_test)

3909/3909 [==============================] - 10s 2ms/step - loss: 0.4414 - accuracy: 0.7774


[0.44135212898254395, 0.7773798108100891]

In [ ]:
y_test_arg=np.argmax(y_test,axis=1)

In [ ]:
Y_pred = np.argmax(ann.predict(X_test),axis=1)

3909/3909 [==============================] - 6s 2ms/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_arg, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11881
           1       0.96      0.87      0.91      7058
           2       0.00      0.00      0.00     11055
           3       0.33      0.84      0.48     11114
           4       0.91      0.93      0.92     24020
           5       0.86      0.74      0.80     36774
           6       1.00      0.97      0.98      8094
           7       0.91      0.58      0.71      4438
           8       0.86      0.93      0.89     10649

    accuracy                           0.78    125083
   macro avg       0.76      0.76      0.74    125083
weighted avg       0.78      0.78      0.76    125083

